In [3]:
my_packages <- c("data.table", "tidyverse",
                 "Seurat", "scCATCH")
pacman::p_load(char = my_packages)

tmp <- list()

# Load data

In [6]:
mat_test <- Matrix::readMM("temp/adata_hvg_singler.mtx")

In [7]:
mat_test_barcode <- fread("temp/adata_hvg_scibet_cellbarcode.csv", header = F)$V1
mat_test_gene <- fread("temp/adata_hvg_scibet_gene.csv", header = F)$V1
mat_test_cluster <- fread("temp/adata_leiden.csv", header = F)$V1

In [ ]:
rownames(mat_test) <- mat_test_barcode
colnames(mat_test) <- mat_test_gene
mat_test <- Matrix::t(mat_test)
mat_test[1:5,1:5]

In [4]:
cv2 <- fread("Data/SCSA/cmarkerv2.csv")

In [5]:
scmatch_marker <- data.frame(
  species = cv2$speciesType,
  tissue = cv2$tissueType,
  cancer = ifelse(grepl("Cancer", cv2$cellType), "Cancer", "Normal"),
  condition = cv2$cellType,
  subtype1 = NA,
  subtype2 = NA,
  subtpye3 = NA,
  celltype = cv2$cellName,
  gene = cv2$gene,
  resource = cv2$markerResource,
  pmid = cv2$PMID 
)

In [17]:
scmatch_marker_sel <- scmatch_marker[scmatch_marker$species %in% "Human" & scmatch_marker$tissue %in% "Stomach",]
dim(scmatch_marker_sel)

[1] 1175   11

# scCATCH

In [21]:
mat_test <- as.matrix(mat_test)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 18.6 GiB”


In [24]:
mat_test_cluster <- as.character(mat_test_cluster)

In [25]:
scCATCH_obj <- createscCATCH(data = mat_test, 
                             cluster = mat_test_cluster)
scCATCH_obj1 <- findmarkergene(object = scCATCH_obj, species = "Human", marker = cellmatch, tissue = "Stomach")
scCATCH_obj2 <- findmarkergene(object = scCATCH_obj, species = "Human", if_use_custom_marker = TRUE, marker = scmatch_marker_sel)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 18.6 GiB”
There are 136 potential marker genes in CellMatch database for Human on Stomach.

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 18.6 GiB”


In [27]:
scCATCH_obj1 <- findcelltype(scCATCH_obj1)


In [31]:
scCATCH_obj1@celltype

cluster,cluster_marker,cell_type,celltype_score,celltype_related_marker,PMID
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
12,ASPM,Progenitor Cell,0.58,ASPM,26178168
14,"MAML2, OLR1, RASSF3",Pit Progenitor Cell,0.61,"OLR1, RASSF3",29802404
7,"CFH, PPP1R14A",Progenitor Cell,0.58,"CFH, PPP1R14A",29802404
4,BCL11A,Progenitor Cell,0.50,BCL11A,29802404
11,"ANXA1, SLC24A3",Progenitor Cell,0.58,"ANXA1, SLC24A3",29802404
9,TCIM,Progenitor Cell,0.50,TCIM,29802404


# Save